In [1]:
import keras 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from format_data import get_data, data_pregen, data_gen
from models import SklearnModel

Using TensorFlow backend.


In [ ]:
# Parameters
first_path = "data/data.h5"
second_path = "data/asu.tsv"

flatten = True
seed = 0
data_path = "data/"

In [ ]:
# Converting the data from data.h5 and asu.tsv into training and testing data.
# Coordinates from data.h5 and asu.tsv are converted into Skycoords.
# The two are cross-matched. Sources within five arcsec radius of each other are assumed to be the same object.
# asu sources are extracted from the data.h5 sources, and formated into class-wise balanced training set.
# The remainder are formated into class-wise balanced testing set.

train_x, train_y, test_x, test_y = get_data(first_path, second_path, 
                                            flip_train_test=False, 
                                            dim=256, conf_matrix=False, seed=seed)

In [ ]:
# Consrtuct the data generator to randomly Augment data
datagen = data_gen(rotation_range=180, zoom_range=0.2, shift_range = 0.0, flip=True)

In [ ]:
# Pre-generating data for sklearn classifer.
pre_train_x, pre_train_y, pre_test_x, pre_test_y = data_pregen(train_x, train_y, test_x, test_y, datagen=datagen, 
                                                               batch_size=20, nb_epoch=5, seed=seed)

In [ ]:
# Instatising Logistic Regression (any other Sklearn Classifer can be used - with the same input data format)
logistic = SklearnModel(LogisticRegression, name="logistic", data_path=data_path, seed=seed)

In [ ]:
# Dictionary of model parameters
params = {'solver': 'liblinear', 'max_iter':100, 'multi_class':'ovr', 'verbose':0}

# Training classifer
logistic = logistic.train(pre_train_x, pre_train_y, params)

In [ ]:
# Loading model
logistic_model = logistic.load()

In [ ]:
# Testing model (won't give results as data is wrong - see readme)
score = logistic_model.predict(pre_test_x)

In [ ]:
score.mean()